In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

# WordCloud and matplotlib for visualization
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## TO DO

- assigning keywords to missing keyword labels - DONE
- remove symbols (#, ".. - DONE
- remove links - DONE
- validation
- make slides - IN PROCESS
- TFIDF, LSA, LSTM / RNNs, the list is long
- try different classifiers
- reading discussion board for inspiration


In [26]:
train = pd.read_csv("preprocessed_train_data.csv", index_col = 0)
test = pd.read_csv("preprocessed_test_data.csv", index_col = 0)

trained_tweets = train['keyword']+train['text']
test_tweets = test['keyword']+test['text']

In [27]:
trained_tweets.head()

0    earthquak  deed reason earthquak may allah for...
1     forest fire  forest fire near la rang ask canada
2    evacu  resid ask shelter place notifi offic ev...
3    wildfir  peopl receiv wildfir evacu order cali...
4    wildfir  got sent photo rubi alaska smoke wild...
dtype: object

In [28]:
test_tweets.head()

0                      crash  happen terribl car crash
1    earthquak  heard earthquak differ citi stay sa...
2    fire  forest fire spot pond gees flee across s...
3            apocalyps  apocalyps light spokan wildfir
4          typhoon  typhoon soudelor kill china taiwan
dtype: object

# Encoding and Vectorizers

In [6]:
# import category_encoders as ce

# # Target encoding
# features = ['keyword']
# encoder = ce.TargetEncoder(cols=features)
# encoder.fit(train[features],train['target'])

# train = train.join(encoder.transform(train[features]).add_suffix('_target'))
# test = test.join(encoder.transform(test[features]).add_suffix('_target'))

In [29]:
X_train, X_test, y_train, y_test = train_test_split(trained_tweets, train['target'].values, 
        train_size = 0.80, test_size = 0.20, random_state = 12, shuffle=True) # We have split train-test dataset using 80:20 ratio

# TFIDF

In [30]:
#precompute vectorized representations
word_vectorizer = TfidfVectorizer(
    analyzer='word',
    stop_words='english',
    ngram_range=(1, 3),
    lowercase=True,
    min_df=5,
    max_features=30000)

char_vectorizer = TfidfVectorizer(
    analyzer='char',
    stop_words='english',
    ngram_range=(3, 6),
    lowercase=True,
    min_df=5,
    max_features=50000)

vectorizer = FeatureUnion([('word_vectorizer', word_vectorizer),  ('char_vectorizer', char_vectorizer)])
vectorizer.fit(X_train)

X_train_vectors = vectorizer.transform(X_train).toarray()
X_test_vectors = vectorizer.transform(X_test).toarray()
print(X_train_vectors.shape, X_test_vectors.shape)

#X_train_text = X_train.tolist()
#X_test_text = X_test.tolist()

/home/neptune/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:520: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'stop_words' will not be used"


(6090, 48864) (1523, 48864)


In [31]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

## Tokenize the sentences
max_features=50000
maxlen=25
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train.tolist())
X_train = tokenizer.texts_to_sequences(X_train.tolist())
X_test = tokenizer.texts_to_sequences(X_test.tolist())

## Pad the sentences 
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [32]:
X_train

array([[   0,    0,    0, ...,  353,  716,  838],
       [   0,    0,    0, ...,   39,  283,  403],
       [   0,    0,    0, ..., 1614,    9, 2313],
       ...,
       [   0,    0,    0, ..., 1496,  947,  310],
       [   0,    0,    0, ...,  364,  513, 1019],
       [   0,    0,    0, ...,  213,   73,  729]], dtype=int32)

# Recurrent Neural Networks

In [11]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout, Embedding
from tensorflow.keras import backend as K

vocab_dim = X_train.max() + 1
embed_dim = int(np.sqrt(vocab_dim))
print(vocab_dim, embed_dim)
model = Sequential([
    Embedding(input_dim=vocab_dim, output_dim=embed_dim),
    SimpleRNN(units=embed_dim, return_sequences=False),
    Dense(1, activation='sigmoid')
])
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

10401 101
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
history = model.fit(
    x=X_train, y=y_train, batch_size=64, epochs=5, verbose=True, validation_data=(X_test, y_test), shuffle=True
)
K.clear_session()

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 2s 293us/sample - loss: 0.6218 - acc: 0.6399 - val_loss: 0.5046 - val_acc: 0.7774
Epoch 2/5
6090/6090 [==============================] - 1s 237us/sample - loss: 0.2555 - acc: 0.9138 - val_loss: 0.5880 - val_acc: 0.7557
Epoch 3/5
6090/6090 [==============================] - 1s 231us/sample - loss: 0.1117 - acc: 0.9660 - val_loss: 0.6161 - val_acc: 0.7787
Epoch 4/5
6090/6090 [==============================] - 1s 227us/sample - loss: 0.0760 - acc: 0.9768 - val_loss: 0.6614 - val_acc: 0.7505
Epoch 5/5
6090/6090 [==============================] - 1s 226us/sample - loss: 0.0689 - acc: 0.9782 - val_loss: 0.6455 - val_acc: 0.7459


In [146]:
model = Sequential([
    Embedding(input_dim=vocab_dim, output_dim=embed_dim),
    SimpleRNN(units=embed_dim, return_sequences=True),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)
model._make_predict_function()

In [147]:
history = model.fit(
    x=X_train, y=y_train, batch_size=64, epochs=10, verbose=True, validation_data=(X_test, y_test), shuffle=True
)

Train on 6090 samples, validate on 1523 samples
Epoch 1/10
6090/6090 [==============================] - 2s 310us/sample - loss: 0.6203 - acc: 0.6609 - val_loss: 0.5377 - val_acc: 0.7492
Epoch 2/10
6090/6090 [==============================] - 2s 260us/sample - loss: 0.3687 - acc: 0.8494 - val_loss: 0.4805 - val_acc: 0.7853
Epoch 3/10
6090/6090 [==============================] - 1s 241us/sample - loss: 0.2050 - acc: 0.9266 - val_loss: 0.5181 - val_acc: 0.7781
Epoch 4/10
6090/6090 [==============================] - 1s 239us/sample - loss: 0.1311 - acc: 0.9594 - val_loss: 0.5580 - val_acc: 0.7708
Epoch 5/10
6090/6090 [==============================] - 1s 236us/sample - loss: 0.0914 - acc: 0.9722 - val_loss: 0.5924 - val_acc: 0.7781
Epoch 6/10
6090/6090 [==============================] - 1s 237us/sample - loss: 0.0716 - acc: 0.9772 - val_loss: 0.7271 - val_acc: 0.7702
Epoch 7/10
6090/6090 [==============================] - 2s 264us/sample - loss: 0.0625 - acc: 0.9793 - val_loss: 0.7195 - va

In [152]:
test = tokenizer.texts_to_sequences(test_tweets.tolist())
test = pad_sequences(test, maxlen=maxlen)
test

array([[   0,    0,    0, ..., 2236,   63,   14],
       [   0,    0,    0, ...,  573, 1397,  242],
       [   0,    0,    0, ...,  877,  623,  120],
       ...,
       [   0,    0,    0, ...,  661,   37,  825],
       [   0,    0,    0, ...,   60,  284, 1583],
       [   0,    0,    0, ...,   25,   96, 4333]], dtype=int32)

In [153]:
predictions = model.predict(test)
K.clear_session()

In [154]:
predictions

array([[0.9865468 ],
       [0.9906554 ],
       [0.998838  ],
       ...,
       [0.99467707],
       [0.67777115],
       [0.9985507 ]], dtype=float32)

In [157]:
pre = []
for p in predictions:
    if p > 0.5:
        pre.append(1)
    else:
        pre.append(0)
pre

[1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,


In [158]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission["target"] = pre
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [159]:
sample_submission.to_csv("submission_nn.csv", index=False)

# Bidirectional RNN

In [16]:
from tensorflow.keras.layers import Bidirectional

model = Sequential([
    Embedding(input_dim=vocab_dim, output_dim=embed_dim),
    Bidirectional(SimpleRNN(units=embed_dim, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
history = model.fit(
    x=X_train, y=y_train, batch_size=64, epochs=5, verbose=True, validation_data=(X_test, y_test), shuffle=True
)
K.clear_session()

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 3s 427us/sample - loss: 0.5701 - acc: 0.7020 - val_loss: 0.4988 - val_acc: 0.7741
Epoch 2/5
6090/6090 [==============================] - 2s 375us/sample - loss: 0.3354 - acc: 0.8683 - val_loss: 0.4942 - val_acc: 0.7905
Epoch 3/5
6090/6090 [==============================] - 2s 376us/sample - loss: 0.1978 - acc: 0.9304 - val_loss: 0.5356 - val_acc: 0.7708
Epoch 4/5
6090/6090 [==============================] - 2s 395us/sample - loss: 0.1261 - acc: 0.9571 - val_loss: 0.5876 - val_acc: 0.7387
Epoch 5/5
6090/6090 [==============================] - 2s 383us/sample - loss: 0.0964 - acc: 0.9649 - val_loss: 0.7050 - val_acc: 0.7571


# LSTM

In [160]:
from tensorflow.keras.layers import LSTM

model = Sequential([
    Embedding(input_dim=vocab_dim, output_dim=embed_dim),
    LSTM(units=embed_dim, return_sequences=False),
    Dense(1, activation='sigmoid')
])
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [161]:
history = model.fit(
    x=X_train, y=y_train, batch_size=64, epochs=5, verbose=True, validation_data=(X_test, y_test), shuffle=True
)
#K.clear_session()

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 4s 585us/sample - loss: 0.5630 - acc: 0.6967 - val_loss: 0.4717 - val_acc: 0.7879
Epoch 2/5
6090/6090 [==============================] - 3s 513us/sample - loss: 0.3189 - acc: 0.8718 - val_loss: 0.5108 - val_acc: 0.7859
Epoch 3/5
6090/6090 [==============================] - 3s 520us/sample - loss: 0.2077 - acc: 0.9189 - val_loss: 0.6206 - val_acc: 0.7833
Epoch 4/5
6090/6090 [==============================] - 3s 530us/sample - loss: 0.1410 - acc: 0.9507 - val_loss: 0.7255 - val_acc: 0.7748
Epoch 5/5
6090/6090 [==============================] - 3s 537us/sample - loss: 0.1072 - acc: 0.9647 - val_loss: 0.8032 - val_acc: 0.7551


In [162]:
predictions = model.predict(test)
K.clear_session()

In [163]:
predictions

array([[0.696366  ],
       [0.7708965 ],
       [0.99975467],
       ...,
       [0.9199234 ],
       [0.66180515],
       [0.9808642 ]], dtype=float32)

In [164]:
pre = []
for p in predictions:
    if p > 0.5:
        pre.append(1)
    else:
        pre.append(0)
pre

[1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [165]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission["target"] = pre
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [166]:
sample_submission.to_csv("submission_lstm.csv", index=False)